In [263]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [264]:
Train = pd.read_csv(r"C:\Users\Dell\Documents\GitHub\regression-predict-api-template\utils\data\train_data.csv")
Riders = pd.read_csv(r'C:\Users\Dell\Documents\GitHub\regression-predict-api-template\utils\data\Riders.csv')
Test = pd.read_csv(r"C:\Users\Dell\Documents\GitHub\regression-predict-api-template\utils\data\test_data.csv")

In [265]:
Test = Test.merge(Riders, how='left', on='Rider Id')

In [266]:
import json
v =Test.iloc[1].to_json()
feature_vector_dict = json.loads(v)
Test = pd.DataFrame.from_dict([feature_vector_dict])

In [267]:
Test

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_21660,User_Id_1329,Bike,3,Business,31,5,12:16:49 PM,31,5,...,None,-1.279518,36.823809,-1.273056,36.811298,Rider_Id_812,4402,1090,14.3,1301


In [268]:
Riders.drop(['No_Of_Orders','Age','No_of_Ratings'], axis=1, inplace=True)

In [269]:
Train.drop(['Order No','User Id','Vehicle Type','Precipitation in millimeters'], axis=1, inplace=True)

In [270]:
Train.drop(['Confirmation - Day of Month','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Day of Month',
            'Arrival at Pickup - Weekday (Mo = 1)','Arrival at Destination - Weekday (Mo = 1)','Pickup - Day of Month',
            'Pickup - Weekday (Mo = 1)','Arrival at Destination - Day of Month','Temperature'], axis=1, inplace=True)

In [271]:
pickup_co= Train[['Pickup Lat', 'Pickup Long']].apply(tuple, axis=1).tolist()
destination_co = Train[['Destination Lat', 'Destination Long']].apply(tuple, axis=1).tolist()

In [272]:
import reverse_geocoder as rg

pickup_re = rg.search(pickup_co, mode=2)
Train['Pickup_city'] = [x['name'] for x in pickup_re]

dest_re = rg.search(destination_co, mode=2)
Train['Destination_city'] = [x['name'] for x in dest_re]

In [273]:
Train['Pickup_to_Destination']= Train[['Pickup_city', 'Destination_city']].agg('-'.join, axis=1)

In [274]:
Train.drop(['Pickup Lat','Pickup Long','Destination Lat','Destination Long',
            'Pickup_city','Destination_city'], axis=1, inplace=True)

In [275]:
def time_to_seconds(column_name):
    Train[column_name] = pd.to_datetime(Train[column_name])
    return (Train[column_name]- pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')

In [276]:
Train['Confirmation - Time']=time_to_seconds('Confirmation - Time')

In [277]:
Train['Con_Time_of_day'] = ['Morning' if x< 43200 else 'Afternoon' for x in Train['Confirmation - Time']]

In [278]:
Train['Platform Type'] = ['Busy' if x == 3  else 'Not_busy' for x in Train['Platform Type']]

In [279]:
Train.drop(['Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Pickup - Time',
            'Arrival at Destination - Time'], axis=1, inplace=True)

In [280]:
Train.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Rider Id,Time from Pickup to Arrival,Pickup_to_Destination,Con_Time_of_day
0,Busy,Business,7,3,3,Rider_Id_646,676,Nairobi-Nairobi,Afternoon
1,Busy,Business,27,3,14,Rider_Id_579,2296,Nairobi-Pumwani,Afternoon
2,Busy,Personal,3,1,3,Rider_Id_146,471,Nairobi-Nairobi,Afternoon
3,Busy,Personal,26,2,15,Rider_Id_345,1335,Pumwani-Nairobi,Afternoon
4,Busy,Business,15,1,4,Rider_Id_206,446,Pumwani-Nairobi,Afternoon


In [281]:
Test.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_21660,User_Id_1329,Bike,3,Business,31,5,12:16:49 PM,31,5,...,None,-1.279518,36.823809,-1.273056,36.811298,Rider_Id_812,4402,1090,14.3,1301


In [282]:
Test.drop(['Order No','User Id','Vehicle Type','Precipitation in millimeters'], axis=1, inplace=True)

In [283]:
Test.drop(['Confirmation - Day of Month','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Day of Month',
            'Arrival at Pickup - Weekday (Mo = 1)','Pickup - Day of Month',
            'Pickup - Weekday (Mo = 1)','Temperature'], axis=1, inplace=True)

In [284]:
pickup_coo= Test[['Pickup Lat', 'Pickup Long']].apply(tuple, axis=1).tolist()
destination_coo = Test[['Destination Lat', 'Destination Long']].apply(tuple, axis=1).tolist()

In [285]:
pickup_ree = rg.search(pickup_coo, mode=2)
Test['Pickup_city'] = [x['name'] for x in pickup_ree]

dest_ree = rg.search(destination_coo, mode=2)
Test['Destination_city'] = [x['name'] for x in dest_ree]

In [286]:
Test['Pickup_to_Destination']= Test[['Pickup_city', 'Destination_city']].agg('-'.join, axis=1)

In [287]:
Test.drop(['Pickup Lat','Pickup Long','Destination Lat','Destination Long',
           'Pickup_city','Destination_city'], axis=1, inplace=True)

In [288]:
def time_to_seconds1(column_name):
    Test[column_name] = pd.to_datetime(Test[column_name])
    return (Test[column_name]- pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')

In [289]:
Test['Confirmation - Time']=time_to_seconds1('Confirmation - Time')

In [290]:
Test['Con_Time_of_day'] = ['Morning' if x< 43200 else 'Afternoon' for x in Test['Confirmation - Time']]

In [291]:
Test['Platform Type'] = ['Busy' if x == 3  else 'Not_busy' for x in Test['Platform Type']]

In [292]:
Test.drop(['Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Pickup - Time'], axis=1, inplace=True)

In [293]:
Test.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Pickup_to_Destination,Con_Time_of_day
0,Busy,Business,31,5,4,Rider_Id_812,4402,1090,14.3,1301,Nairobi-Nairobi,Afternoon


In [294]:
Train['train'] = 1
Test['train'] = 0

In [295]:
combined = pd.concat([Train,Test])

In [296]:
combined = combined.merge(Riders, how='left', on = 'Rider Id')

In [297]:
combined

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Rider Id,Time from Pickup to Arrival,Pickup_to_Destination,Con_Time_of_day,train,No_Of_Orders,Age,Average_Rating_x,No_of_Ratings,Average_Rating_y
0,Busy,Business,7,3,3,Rider_Id_646,676.0,Nairobi-Nairobi,Afternoon,1,NaN,NaN,NaN,NaN,15.0
1,Busy,Business,27,3,14,Rider_Id_579,2296.0,Nairobi-Pumwani,Afternoon,1,NaN,NaN,NaN,NaN,13.3
2,Busy,Personal,3,1,3,Rider_Id_146,471.0,Nairobi-Nairobi,Afternoon,1,NaN,NaN,NaN,NaN,13.5
3,Busy,Personal,26,2,15,Rider_Id_345,1335.0,Pumwani-Nairobi,Afternoon,1,NaN,NaN,NaN,NaN,14.2
4,Busy,Business,15,1,4,Rider_Id_206,446.0,Pumwani-Nairobi,Afternoon,1,NaN,NaN,NaN,NaN,13.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18016,Busy,Business,14,2,6,Rider_Id_448,685.0,Nairobi-Nairobi,Afternoon,1,NaN,NaN,NaN,NaN,14.6
18017,Busy,Business,14,4,15,Rider_Id_39,1962.0,Nairobi-Nairobi,Morning,1,NaN,NaN,NaN,NaN,13.3
18018,Not_busy,Personal,7,1,15,Rider_Id_789,2428.0,Nairobi-Pumwani,Afternoon,1,NaN,NaN,NaN,NaN,14.1
18019,Busy,Business,31,5,14,Rider_Id_211,1720.0,Nairobi-Nairobi,Morning,1,NaN,NaN,NaN,NaN,13.3


In [298]:
combined = combined[['Platform Type', 'Distance (KM)', 'Rider Id','Personal or Business','Pickup_to_Destination',
                     'Con_Time_of_day','Placement - Day of Month',
                     'Placement - Weekday (Mo = 1)','train','Time from Pickup to Arrival']]

In [299]:
combined.columns = [col.replace(' ','_') for col in combined.columns]

combined.columns = [col.replace('(','') for col in combined.columns]
combined.columns = [col.replace(')','') for col in combined.columns]


In [300]:
combined

,Platform_Type,Distance_KM,Rider_Id,Personal_or_Business,Pickup_to_Destination,Con_Time_of_day,Placement_-_Day_of_Month,Placement_-_Weekday_Mo_=_1,train,Time_from_Pickup_to_Arrival
0,Busy,3,Rider_Id_646,Business,Nairobi-Nairobi,Afternoon,7,3,1,676.0
1,Busy,14,Rider_Id_579,Business,Nairobi-Pumwani,Afternoon,27,3,1,2296.0
2,Busy,3,Rider_Id_146,Personal,Nairobi-Nairobi,Afternoon,3,1,1,471.0
3,Busy,15,Rider_Id_345,Personal,Pumwani-Nairobi,Afternoon,26,2,1,1335.0
4,Busy,4,Rider_Id_206,Business,Pumwani-Nairobi,Afternoon,15,1,1,446.0
...,...,...,...,...,...,...,...,...,...,...
18016,Busy,6,Rider_Id_448,Business,Nairobi-Nairobi,Afternoon,14,2,1,685.0
18017,Busy,15,Rider_Id_39,Business,Nairobi-Nairobi,Morning,14,4,1,1962.0
18018,Not_busy,15,Rider_Id_789,Personal,Nairobi-Pumwani,Afternoon,7,1,1,2428.0
18019,Busy,14,Rider_Id_211,Business,Nairobi-Nairobi,Morning,31,5,1,1720.0


In [301]:
df_dummies = pd.get_dummies(combined,columns=['Platform_Type','Rider_Id','Pickup_to_Destination',
                                              'Con_Time_of_day','Placement_-_Day_of_Month','Placement_-_Weekday_Mo_=_1','Personal_or_Business'], drop_first=True)

In [302]:
df_dummies.head()

,Distance_KM,train,Time_from_Pickup_to_Arrival,Platform_Type_Not_busy,Rider_Id_Rider_Id_1,Rider_Id_Rider_Id_10,Rider_Id_Rider_Id_100,Rider_Id_Rider_Id_101,Rider_Id_Rider_Id_102,Rider_Id_Rider_Id_103,...,Placement_-_Day_of_Month_29,Placement_-_Day_of_Month_30,Placement_-_Day_of_Month_31,Placement_-_Weekday_Mo_=_1_2,Placement_-_Weekday_Mo_=_1_3,Placement_-_Weekday_Mo_=_1_4,Placement_-_Weekday_Mo_=_1_5,Placement_-_Weekday_Mo_=_1_6,Placement_-_Weekday_Mo_=_1_7,Personal_or_Business_Personal
0,3,1,676.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,14,1,2296.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,1,471.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,15,1,1335.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,4,1,446.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [303]:
Test2 = df_dummies[df_dummies['train']== 0]
Test2.drop(['train'],axis=1,inplace=True)

Train2 = df_dummies[df_dummies['train']==1]
Train2.drop(['train'],axis=1, inplace=True)

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [304]:
Test2.drop(['Time_from_Pickup_to_Arrival'], axis=1,inplace=True)

In [320]:
Test2.to_csv('t2.csv',index=False)

In [306]:
column_titles = [col for col in Train2.columns if col !=
                 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
Train3 = Train2.reindex(columns=column_titles)

In [307]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [308]:
x = Train3.iloc[:,:-1]
y = Train3.iloc[:,-1]

In [309]:
scaler = StandardScaler()
x.iloc[:,:2] = scaler.fit_transform(x.iloc[:,:2])

In [310]:
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    shuffle=False)

In [311]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(X_train, y_train)

Ridge()

In [312]:
train_ridge = ridge.predict(X_train)
test_ridge = ridge.predict(X_test)

In [313]:
from math import sqrt
from  sklearn import metrics
print('Ridge :', sqrt(metrics.mean_squared_error(y_train, train_ridge)))
print('Ridge :', sqrt(metrics.mean_squared_error(y_test, test_ridge)))
       

Ridge : 716.1313375006761
Ridge : 777.1327936114299


In [314]:
Test2.iloc[:,:2] = scaler.transform(Test2.iloc[:,:2])

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [315]:
r = metrics.r2_score(y_train, train_ridge)
Testr = ridge.predict(Test2)
Testr

array([975.15124909])

In [316]:
Test_res = pd.DataFrame(Testr, columns=['Time from Pickup to Arrival'])
Tests = pd.read_csv('Test.csv')
Tests['Time from Pickup to Arrival'] = Test_res

In [317]:
Submission2 = Tests[['Order No','Time from Pickup to Arrival']].copy()

In [318]:
Submission2.to_csv('Submission5.csv',index=False)

In [319]:
import pickle

model_save_path = 'mlr_Ridge_model5.pkl'

with open(model_save_path, 'wb') as file:
    pickle.dump(ridge, file)